# Get Territory raw data data from google cloud storage

In [1]:
from google.cloud import storage
import json

In [2]:
# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket('opengeokpi-bucket')

In [37]:
# get blobs
blob_zones = bucket.blob('raw/territory/geozones-france-2019.json')

In [38]:
# Download the contents of the blob as a string and then parse it using json.loads() method
data_zones = json.loads(blob_zones.download_as_string(client=None))

# Prepare data
## Regions

In [39]:
import geopandas as gpd
from shapely import wkt

In [53]:
# convert geojson to geopandadataframe
gdf_zones = gpd.GeoDataFrame.from_features(data_zones["features"])

In [54]:
# set crs
gdf_zones = gdf_zones.set_crs(epsg=4326)

In [56]:
# select regions
gdf_zones_regions =  gdf_zones[gdf_zones.level == 'fr:region']

In [64]:
gdf_zones.level.unique()

array(['country', 'country-group', 'country-subset', 'fr:arrondissement',
       'fr:collectivite', 'fr:commune', 'fr:departement', 'fr:epci',
       'fr:region'], dtype=object)

In [57]:
# subset columns
gdf_zones_regions = gdf_zones_regions[['geometry','level','code','name','population','area']]

In [59]:
# save region geojson
with open('data/territory/gdf_zones_regions.geojson', 'w') as f:
    f.write(gdf_zones_regions.to_json())

In [60]:
# upload in google storage
blob = bucket.blob('refine/territory/regions.json')

region_geojson_file  = 'data/territory/gdf_zones_regions.geojson'
blob.upload_from_filename(region_geojson_file)

print("File uploaded to {}.".format(bucket.name))

File uploaded to opengeokpi-bucket.


In [51]:
# reload data to verify

# get blobs
#blob_regions = bucket.blob('raw/territoire/regions.json')

# Download the contents of the blob as a string and then parse it using json.loads() method
#data_regions = json.loads(blob_regions.download_as_string(client=None))

# convert geojson to geopandadataframe
#gdf_regions = gpd.GeoDataFrame.from_features(data_regions["features"])
#gdf_regions.head()

## Departements

In [52]:
# select departement
gdf_zones_departement =  gdf_zones[gdf_zones.level == 'fr:departement']

,geometry,level,code,name,wikidata,wikipedia,population,area,flag,keys,parents,id,color,website,validity,blazon,successors,ancestors
43705,"MULTIPOLYGON (((5.80669 46.04423, 5.80887 46.0...",fr:departement,01,Ain,Q3083,fr:Ain_(d%C3%A9partement),631877.0,5762.0,Flag_of_Ain.svg,"{'insee': '01', 'nuts3': 'fr711', 'geonames': ...","[country:fr, country-group:ue, country-group:w...",fr:departement:01@1860-07-01,#536eac,http://www.ain.fr/,"{'start': '1860-07-01', 'end': None}",Blason_d%C3%A9partement_fr_Ain.svg,NaN,NaN
43706,"MULTIPOLYGON (((3.07681 49.35563, 3.07741 49.3...",fr:departement,02,Aisne,Q3093,fr:Aisne_(d%C3%A9partement),538659.0,7369.0,Drapeau_fr_d%C3%A9partement_Aisne.svg,"{'insee': '02', 'nuts3': 'fr221', 'geonames': ...","[country:fr, country-group:ue, country-group:w...",fr:departement:02@1860-07-01,#58783a,http://www.aisne.pref.gouv.fr/,"{'start': '1860-07-01', 'end': None}",Blason_d%C3%A9partement_fr_Aisne.svg,NaN,NaN
43707,"MULTIPOLYGON (((3.23258 46.06980, 3.22433 46.0...",fr:departement,03,Allier,Q3113,fr:Allier_(d%C3%A9partement),341613.0,7340.0,Drapeau_fr_d%C3%A9partement_Allier.svg,"{'insee': '03', 'nuts3': 'fr721', 'geonames': ...","[country:fr, country-group:ue, country-group:w...",fr:departement:03@1860-07-01,#e0dc6c,http://www.allier.pref.gouv.fr/,"{'start': '1860-07-01', 'end': None}",Blason_comte_fr_Clermont_(Bourbon).svg,NaN,NaN
43708,None,fr:departement,04,Basses-Alpes,NaN,NaN,NaN,NaN,NaN,{'insee': '04'},"[country:fr, country-group:ue, country-group:w...",fr:departement:04@1860-07-01,#73e06c,NaN,"{'start': '1860-07-01', 'end': '1970-04-12'}",NaN,[fr:departement:04@1970-04-13],NaN
43709,"MULTIPOLYGON (((5.65458 43.82504, 5.64606 43.8...",fr:departement,04,Alpes-de-Haute-Provence,Q3131,fr:Alpes-de-Haute-Provence,161799.0,6925.0,NaN,"{'insee': '04', 'nuts3': 'fr821', 'geonames': ...","[country:fr, country-group:ue, country-group:w...",fr:departement:04@1970-04-13,#40bfa4,http://www.alpes-de-haute-provence.pref.gouv.fr/,"{'start': '1970-04-13', 'end': None}",NaN,NaN,[fr:departement:04@1860-07-01]


In [61]:
# subset columns
gdf_zones_departement = gdf_zones_departement[['geometry','level','code','name','population','area']]

In [62]:
# save region geojson
with open('data/territory/gdf_zones_departement.geojson', 'w') as f:
    f.write(gdf_zones_departement.to_json())

In [63]:
# upload in google storage
blob = bucket.blob('refine/territory/departements.json')

departement_geojson_file  = 'data/territory/gdf_zones_departement.geojson'
blob.upload_from_filename(departement_geojson_file)

print("File uploaded to {}.".format(bucket.name))

File uploaded to opengeokpi-bucket.


## communes

In [65]:
# select communes
gdf_zones_communes =  gdf_zones[gdf_zones.level == 'fr:commune']

In [67]:
# subset columns
gdf_zones_communes = gdf_zones_communes[['geometry','level','code','name','population','area']]

In [68]:
# save region geojson
with open('data/territory/gdf_zones_communes.geojson', 'w') as f:
    f.write(gdf_zones_communes.to_json())

In [69]:
# upload in google storage
blob = bucket.blob('refine/territory/communes.json')

communes_geojson_file  = 'data/territory/gdf_zones_communes.geojson'
blob.upload_from_filename(communes_geojson_file)

print("File uploaded to {}.".format(bucket.name))

File uploaded to opengeokpi-bucket.
